In [1]:
import os
import sys
import glob
import cv2
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from mtcnn.mtcnn import MTCNN
from numpy import expand_dims
from numpy import asarray


import keras
from keras import backend as K
from keras import optimizers
import keras.utils
from keras.models import Sequential, Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers import Input, Dense, Flatten, Dropout, Activation, Lambda, Permute, Reshape
from keras.layers import Convolution2D, ZeroPadding2D, MaxPooling2D
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras_vggface.vggface import VGGFace
from keras.applications.resnet50 import preprocess_input
from keras.applications.resnet50 import ResNet50
from keras.optimizers import SGD

Using TensorFlow backend.
/home/n_k_rabeh/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/n_k_rabeh/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/n_k_rabeh/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/n_k_rabeh/.local/lib/python3.6/site-packages/tenso

Define functions needed later.

In [2]:
def extract_face1(filename, required_size=(224, 224)):
    # load image from file
    if type(filename)==str:
        pixels = cv2.imread(filename) # le parmètre entré est le path d'un fichier image    
    else:
        pixels=filename.astype('uint8') # le parmètre entré estun fichier image
    if np.shape(pixels)==(224,224,3):
        face_array=pixels[:,:,::-1]
        return face_array
    # create the detector, using default weights
    detector = MTCNN()
    # detect faces in the image
    results = detector.detect_faces(pixels)
    if results == []:
        return(results)
    # extract the bounding box from the first face
    x1, y1, width, height = results[0]['box']
    x2, y2 = x1 + width, y1 + height
    # extract the face
    face = pixels[y1:y2, x1:x2]
    # resize pixels to the model size
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = np.asarray(image)
    face_array=face_array[:,:,::-1]
    return face_array

Set the different variables needed for the model.

In [4]:
#Set training and validation directory locations
train_dir="./Dataset/train/"
val_dir="./Dataset/validation/"
#image used for training and validation have fixed dimensions
img_width, image_height = 224, 224

batch_size = 2

if your train you model on jupyter, make sure to delete hidden files in Dataset/train and validation. ( e.g. '.ipynb_checkpoints').\
command is : rm -r .ipynb_checkpoints

### Model training

In [ ]:
#Get the class labels based on train directory
labels = os.listdir(train_dir)
labels=[labels[i].split('/')[-1] for i in range(len(labels))]

np.save(open('labels.npy', 'wb'),labels)
total_classes = len(labels)

#Number of training and validation images.
nTrain = sum([len(files) for r,d, files in os.walk(train_dir)])
nValidation = sum([len(files) for r,d, files in os.walk(val_dir)])

#check the labels of the different classes
print(labels)

In [7]:
# create the model 

#Generate data (training and validation)
datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
generator = datagen.flow_from_directory(
    train_dir,
    target_size = (img_width, image_height),
    batch_size = batch_size,
    class_mode = "categorical",
    shuffle = True)

generator_val = datagen.flow_from_directory(
    val_dir,
    target_size = (img_width, image_height),
    batch_size = batch_size,
    class_mode ="categorical",
    shuffle = True)


#build base model 
vgg_model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3),  pooling=None )

# add missing layers
last_layer = vgg_model.output
x = Flatten(name='flatten')(last_layer)
x = Dense(total_classes, activation=None, name='classifier')(x)
out = Activation('softmax')(x)


# this is the model we will train
model = Model(inputs=vgg_model.input, outputs=out)

# We retrain all layers except the first layer
# i.e. freeze only first layer
model.layers[0].trainable = False
sgd = optimizers.SGD(lr = 0.001, decay = 1e-6, momentum = 0.9, nesterov = True)

# compile the model 
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics= [ "acc"])

#Train
model.fit_generator(generator, epochs=15,validation_data=generator_val)

Found 1288 images belonging to 7 classes.
Found 244 images belonging to 7 classes.



Epoch 1/15
644/644 [==============================] - 440s 684ms/step - loss: 2.1527 - acc: 0.4092 - val_loss: 1.6246 - val_acc: 0.4754
Epoch 2/15
644/644 [==============================] - 433s 672ms/step - loss: 1.0158 - acc: 0.6483 - val_loss: 1.7956 - val_acc: 0.7254
Epoch 3/15
644/644 [==============================] - 631s 979ms/step - loss: 0.6894 - acc: 0.7787 - val_loss: 0.0320 - val_acc: 0.5328
Epoch 4/15
644/644 [==============================] - 490s 761ms/step - loss: 0.5038 - acc: 0.8300 - val_loss: 1.3878 - val_acc: 0.7172
Epoch 5/15
644/644 [==============================] - 429s 666ms/step - loss: 0.3803 - acc: 0.8680 - val_loss: 1.5976 - val_acc: 0.7295
Epoch 6/15
644/644 [==============================] - 427s 664ms/step - loss: 0.2447 - acc: 0.9255 - val_loss: 1.0758 - val_acc: 0.7869
Epoch 7/15
644/644 [==============================] - 426s 662ms/step - loss: 0.1895 - acc: 0.9363

In [12]:
Personnes=[]
for X in os.listdir('./Dataset/train/'):
    for y in os.listdir('./Dataset/train/'+X):
        filename = './Dataset/train/'+X+'/'+y
        Personnes.append(filename)
        break
personnes=[personne.split('/')[3] for personne in Personnes]

In [ ]:
# find ordered classes
classe=[0]*total_classes
for X in Personnes:
    img = image.load_img(X, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    predictions=model.predict(x)
    print(X,predictions[0])
  
    classe[np.argmax(predictions[0])]=X.split('/')[3]
print("les classe sont : ", classe)

In [15]:
model.save('./model/Model.h5')